In [226]:
import pandas as pd

players = pd.read_csv("../../data/raw/running_backs.csv")
teams = pd.read_csv("../../data/raw/team_names.csv")
caps = pd.read_csv("../../data/processed/CapHit.csv")

### Clean CapInfo data:

Remove empty rows at bottom:

In [227]:
caps = caps.dropna().drop(columns="Unnamed: 0")

Convert full team names to their abbreviations:

In [228]:
teamConversion = dict(zip(list(teams["name"]), list(teams["abbr"])))

caps["Team"] = [teamConversion[name] for name in list(caps["Team"])]

In [230]:
def to_int(vals):
    return [int(val) for val in vals]

caps["Year"] = to_int(caps["Year"])
caps["CapHit"] = to_int(caps["CapHit"])
caps["CashSpent"] = to_int(caps["CashSpent"])
caps["BaseSalaryCap"] = to_int(caps["BaseSalaryCap"])

In [232]:
caps.head()

,Year,Player,Team,CapHit,CashSpent,BaseSalaryCap,PercentHit
5,2024,Christian McCaffrey,CAR,16300000,12000000,256000000,0.063672
6,2024,Dalvin Cook,MIN,15601273,12500000,256000000,0.060942
7,2024,Alvin Kamara,NO,14800000,11800000,256000000,0.057813
8,2024,Joe Mixon,CIN,13133205,10383205,256000000,0.051302
9,2024,Ezekiel Elliott,DAL,12600000,10000000,256000000,0.049219


### Combine CapInfo with running_backs:

Find player id from caps data:

In [184]:
# single row data frames:
def player_match(player_data, caps_data):
    return (
        (player_data["year"] == caps_data["Year"]) and 
        (player_data["team"] == caps_data["Team"])
    )


def get_same_name(caps_data):
    return players[players["playername"] == caps_data["Player"].values[0].rstrip()]


def get_player_id(caps_data):
    # get players with the same name:
    same_name = get_same_name(caps_data)
    
    # if can't find any player with the same name
    if len(same_name) == 0:
        return 0
    
    # if there is only one player with that name
    if len(set(same_name["playerid"])) == 1:
        return same_name["playerid"].values[0]
    
    # if there are multiple players with that name
    years = list(caps_data["Year"])
    
    for i in range(len(years)):
        # get players that have the same year
        same_name_year = same_name[same_name["year"] == years[i]]
        
        for j in range(len(same_name_year)):
            # check if were on the same team
            if player_match(same_name_year.iloc[j], caps_data.iloc[i]):
                return same_name_year.iloc[j]["playerid"]
            
    return 0

Combine data functions:

In [254]:
# caps data has duplicate years
def remove_dup_years(df):
    unique = set(df["Year"])
    total = list(df["Year"])
    
    if len(unique) == len(total):
        return df
    
    indexes = [True] * len(total)
    
    # total is in decreasing order so the second one (older one) is removed
    for i in range(1, len(total)):
        if total[i] == total[i - 1]:
            indexes[i] = False
            
    return df[indexes]

# get indexes of player data with same year as caps_data
def get_indexes(player_data, caps_data):
    caps_years = list(caps_data["Year"])
    player_years = list(player_data["year"])
    
    cy_len = len(caps_years)
    py_len = len(player_years)
    
    cy_indexes = [False] * cy_len
    py_indexes = [False] * py_len
    
    for i in range(cy_len):
        if caps_years[i] in player_years:
            cy_indexes[i] = True
    
    for i in range(py_len):
        if player_years[i] in caps_years:
            py_indexes[i] = True
    
    return [player_data[py_indexes], caps_data[cy_indexes]]


# add new columns:
def combine(player_data, caps_data):
    player_data["caphit"] = list(caps_data["CapHit"])
    player_data["cashspent"] = list(caps_data["CashSpent"])
    player_data["basesalarycap"] = list(caps_data["BaseSalaryCap"])
    player_data["percenthit"] = list(caps_data["PercentHit"])
    
    return player_data.drop(columns="Unnamed: 0")

In [262]:
headers = True

for player_name, caps_data in caps.groupby("Player"):     
    player_id = get_player_id(caps_data)
    
    if player_id:
        player_data = players[players["playerid"] == player_id]
        caps_data_no_dup = remove_dup_years(caps_data)
        
        player_data, caps_data = get_indexes(player_data, caps_data_no_dup)
        
        combined = combine(player_data, caps_data)                
        combined.to_csv("../../data/processed/combined.csv", mode='a', header=headers)
        headers = False